### Load libraries and relevant modules into py env

In [1]:
# Import and load relevant libraries
import numpy as np
from kilosort import run_kilosort
from pathlib import Path
import pandas as pd
import os
import glob
from IPython.display import clear_output

### Define Kilosort pipeline

In [3]:
def kikuchi_ks_run(filename):
    # Setup directories and file names
    # Directory for .bin files
    bin_main = Path('C:\\KIKUCHI-LOCAL\\data\\ephys\\bin')
    full_bin_path = bin_main / f"{filename}.dat"

    # Directory for kilosort output
    ks_main = Path('C:\\KIKUCHI-LOCAL\\data\\ephys\\ks')
    full_ks_path = ks_main /f"{filename}"

    # Load in ephys log
    gsheetid = "1_kpK6t0yXWO5wVneRrX4kspHJXAnouSg"
    sheet_name = "agl_t"
    gsheet_url = f"https://docs.google.com/spreadsheets/d/{gsheetid}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    ephysLog = pd.read_csv(gsheet_url)

    # Find session log index
    session_idx = ephysLog[ephysLog['session'] == filename].index[0]

    # Define session specific information\
    # Recording system (for sampling frequency - fs)
    system = ephysLog.at[session_idx, 'sys']
    if system == 'plex': fs = 32000
    else: fs = 24414.0625

    # Electrode configuration
    monkey = ephysLog.at[session_idx, 'monkey']
    if system == 'troy': probe_config = 'C:\\KIKUCHI-LOCAL\\script\\2024-aglt-sequence\\data-extraction\\ks-processing\\probes\\Linear16x2_150_150.mat';
    else: probe_config = 'C:\\KIKUCHI-LOCAL\\script\\2024-aglt-sequence\\data-extraction\\ks-processing\\probes\\Linear16x2_150_200.mat';

    # Define Kilosort parameters
    settings = {'filename': full_bin_path, 'n_chan_bin': 32, 'fs': fs, 'batch_size': 512000, 'nblocks': 0,
                'Th_universal': 9, 'Th_learned': 7, 'tmin': 0, 'tmax': np.inf, 'nt0min': None,
                'dmin': None, 'min_template_size': 10, 'template_sizes': 5, 'nearest_chans': 3,
                'nearest_templates': 3, 'n_templates': 6, 'n_pcs': 6, 'Th_single_ch': 4}

    # Run Kilosort detection and sorting algorithms
    run_kilosort(settings = settings, probe_name = probe_config, results_dir = full_ks_path);

### Define functions to find binary files and determine analysis state

In [4]:
# Function to find all .dat files in the given folder
def find_dat_files(folder_path):
    # Create a pattern for .dat files
    pattern = os.path.join(folder_path, '*.dat')
    
    # Use glob to find all matching files
    dat_files = glob.glob(pattern)
    
    return dat_files


# Function to check if a file exists in the given directory
def file_exists(directory, filename):
    # Create the full path to the file
    file_path = os.path.join(directory, filename)
    
    # Check if the file exists
    return os.path.isfile(file_path)

### Loop extraction algorithm

In [11]:
# Filename for given session
dat_files = find_dat_files('C:\\KIKUCHI-LOCAL\\data\\ephys\\bin')

for file_i in dat_files:
    clear_output(wait=True)
    print(file_i)

    # Get the base name (filename with extension)
    base_name = os.path.basename(file_i)
    filename = os.path.splitext(base_name)[0]

    # Determine if KS output files exist (has KS been run?)
    ks_out_file = 'C:\\KIKUCHI-LOCAL\\data\\ephys\\ks\\' + filename
    ks_flag = file_exists(ks_out_file, 'params.py')

    if ks_flag == 0: kikuchi_ks_run(filename)



C:\KIKUCHI-LOCAL\data\ephys\bin\walt-agl_t-2020-11-30.dat
